# Infer LYSOSOME - part 4

--------------

## OBJECTIVE:  Infer sub-cellular component LYSOSOME  in order to understand interactome 



Dependencies:
The LYSOSOMES  inference rely on the CYTOSOL, which is SOMA&~NUCLEI.  



## IMPORTS

In [1]:

# this needs to be organzied to explain the imports
from pathlib import Path
import os
from collections import defaultdict

import numpy as np
import scipy
from scipy import ndimage as ndi

import napari

# function for core algorithm
import aicssegmentation
from aicssegmentation.core.seg_dot import dot_3d_wrapper, dot_slice_by_slice, dot_2d_slice_by_slice_wrapper, dot_3d
from aicssegmentation.core.pre_processing_utils import ( intensity_normalization, 
                                                         image_smoothing_gaussian_3d,  
                                                         image_smoothing_gaussian_slice_by_slice )
from aicssegmentation.core.utils import topology_preserving_thinning, hole_filling
from aicssegmentation.core.MO_threshold import MO
from aicssegmentation.core.vessel import filament_2d_wrapper, vesselnessSliceBySlice
from aicssegmentation.core.output_utils import   save_segmentation,  generate_segmentation_contour
                                                 
from skimage import filters, img_as_float
from skimage import morphology

from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.morphology import remove_small_objects, binary_closing, ball, disk, dilation, white_tophat, black_tophat   # function for post-processing (size filter)
from skimage.measure import label

# # package for io 
# from aicsimageio import AICSImage

from napari.utils.notebook_display import nbscreenshot



%load_ext autoreload
%autoreload 2
#import .infer_subc.base
from infer_subc.base import *

viewer = None

---------------------

# IMAGE PROCESSING Objective 4:  infer LYSOSOME
> Back to  [OUTLINE: Objective #4](#summary-of-objectives)

## summary of steps (Workflow #1 & #2)

INPUT
- channel  2
- CY mask

PRE-PROCESSING
-  smoothe / remove noise

CORE-PROCESSING
- enhance two classes of "spots"
-  segment objects

POST-PROCESSING
  - filter objects

OUTPUT
- object LYSOSOME 



------------------------
# LOAD RAW IMAGE DATA
Identify path to _raw_ image data and load our example image


In [2]:
##  set up files

data_path = Path( f"{os.getenv('HOME')}/Projects/Imaging/mcz_subcell/data")
czi_img_folder = data_path/"raw"

list_img_files = lambda img_folder,f_type: [os.path.join(img_folder,f_name) for f_name in os.listdir(img_folder) if f_name.endswith(f_type)]

img_file_list = list_img_files(czi_img_folder,'.czi')
print(img_file_list[5])
test_img_name = img_file_list[5]

img_data, meta_dict = read_input_image(test_img_name)

raw_meta_data, ome_types = get_raw_meta_data(meta_dict)

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']

/Users/ahenrie/Projects/Imaging/mcz_subcell/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR3_N04_Unmixed.czi


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


In [3]:
chan_name = 'nuclei'
out_path = data_path / "inferred_objects" 
object_name = 'CY_object'

CY_object, meta_dict_t = read_input_image( out_path/ f"{object_name}.ome.tiff" )



# WORKFLOW #1 

Generally following the Allen Cell Segmenter procedure, but doing more aggressive contrast scaling than their prescribed contrast scaling.

> Using Allen Cell Segmenter LAMP1 [workflow](https://www.allencell.org/cell-observations/category/lamp1).  Examples sourced from: [Notebook](/Users/ahenrie/Projects/Imaging/mcz_subcell/napari/aics-segmentation/lookup_table_demo/playground_lamp1.ipynb) and [script](/Users/ahenrie/Projects/Imaging/mcz_subcell/napari/aics-segmentation/aicssegmentation/structure_wrapper/seg_lamp1.py)

## summary of steps

INPUT
- channel 1
- CYTO mask

PRE-PROCESSING
- contrast scale
- ne-noise and somoothe

CORE PROCESSING
- dot enhancement / segment
- filiment enhancement / segment

POST-PROCESSING
  - fill holes
  - remove small objects


OUTPUT
- mask of LYSOSOME


In [4]:
##########################################################################
# DEFAULT PARAMETERS:
#   note that these parameters are supposed to be fixed for the structure
#   and work well accross different datasets
# default_params = defaultdict(str)

default_params = defaultdict(str, **{
    #"intensity_norm_param" : [0.5, 15]
    "intensity_norm_param" : [0],
    "gaussian_smoothing_sigma" : 1.34,
    "gaussian_smoothing_truncate_range" : 3.0,
    "dot_2d_sigma" : 2,
    "dot_2d_sigma_extra" : 1,
    "dot_2d_cutoff" : 0.025,
    "min_area" : 10,
    "low_level_min_size" :  100,
    "median_filter_size" : 10
})


################################
# calculate a filter dimension for median filtering which considers the difference in scale of Z
z_factor = scale[0]//scale[1]
med_filter_size = 4 #2D 
med_filter_size_3D = (1,med_filter_size,med_filter_size)  # set the scale for a typical median filter
default_params['z_factor'] = z_factor
default_params['scale'] = scale


### INPUT

In [5]:

###################
# INPUT
###################
struct_img_raw = img_data[1,:,:,:].copy()

# DEFAULT PARAMETERS:
intensity_norm_param = [3.5, 15] # from Allen Cell Segmenter LAMP1  workflow
scaling_param = [0]
gaussian_smoothing_sigma = 1.
gaussian_smoothing_truncate_range = 3.0
dot_2d_sigma = 2
dot_2d_sigma_extra = 1
dot_2d_cutoff = 0.025
min_area = 10
low_level_min_size =  100

med_filter_size =3  

gaussian_smoothing_sigma = 1.3
gaussian_smoothing_truncate_range = 3.0

aicssegmentation.core.pre_processing_utils.suggest_normalization_param(struct_img_raw) #  [0., 23]


mean intensity of the stack: 641.5999045901829
the standard deviation of intensity of the stack: 2144.942904845627
0.9999 percentile of the stack intensity is: 49648.38039997965
minimum intensity of the stack: 0
maximum intensity of the stack: 65535
suggested upper range is 23.0, which is 49975.2867160396
suggested lower range is 0.0, which is 641.5999045901829
So, suggested parameter for normalization is [0.0, 23.0]
To further enhance the contrast: You may increase the first value (may loss some dim parts), or decrease the second value(may loss some texture in super bright regions)
To slightly reduce the contrast: You may decrease the first value, or increase the second value


### PRE-PROCESSING

In [6]:
###################
# PRE_PROCESSING
###################

intensity_norm_param = [0, 9] # from Allen Cell Segmenter LAMP1  workflow

# Linear-ish smoothing
raw_lysosomes = intensity_normalization( struct_img_raw ,  scaling_param=intensity_norm_param)

# structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
struct_img = median_filter_slice_by_slice( 
                                                                raw_lysosomes,
                                                                size=med_filter_size  )


structure_img_smooth = image_smoothing_gaussian_slice_by_slice(   struct_img,
                                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                                        truncate_range=gaussian_smoothing_truncate_range,
                                                                                                                    )


# log_img, d = log_transform( structure_img_smooth ) 
# struct_img = intensity_normalization(  log_img  ,  scaling_param=[0] )  

struct_img = structure_img_smooth

### CORE PROCESSING

In [7]:
###################
# CORE_PROCESSING
###################
# dot and filiment enhancement - 2D

################################
## PARAMETERS for this step ##
s2_param = [[5,0.09], [2.5,0.07], [1,0.01]]
################################
bw_spot = dot_2d_slice_by_slice_wrapper(struct_img, s2_param)


################################
## PARAMETERS for this step ##
f2_param = [[1, 0.15]]
################################
bw_filament = filament_2d_wrapper(struct_img, f2_param)
bw = np.logical_or(bw_spot, bw_filament)



### POST-PROCESSING

In [8]:
###################
# POST_PROCESSING
###################

################################
## PARAMETERS for this step ##
fill_2d = True
fill_max_size = 1600
minArea = 15
################################

removed_holes = hole_filling(bw, 0, fill_max_size, fill_2d)

# # 3D
# cleaned_img = remove_small_objects(removed_holes>0, 
#                                                             min_size=minArea, 
#                                                             connectivity=1, 
#                                                             in_place=False)
width = 45  
cleaned_img = size_filter(removed_holes, # wrapper to remove_small_objects which can do slice by slice
                                                         min_size= width**2, 
                                                         method = "3D" ,
                                                         connectivity=1)


#### Visualize with `napari`
Visualize the first-pass segmentation and labeling with `napari`.

In [9]:

if viewer is None:
    viewer = napari.view_image(
        cleaned_img,
        scale=scale
    )
else: 
    viewer.add_image(
        cleaned_img,
        scale=scale
    )

viewer.scale_bar.visible = True
viewer.add_image(
    structure_img_smooth,
    scale=scale
)


<Image layer 'structure_img_smooth' at 0x14a738be0>

### DEFINE `infer_LYSOSOMES` function

In [31]:
##########################
#  infer_LYSOSOMES
##########################
def infer_LYSOSOMES(struct_img,  CY_object, in_params) -> tuple:
    """
    Procedure to infer LYSOSOME from linearly unmixed input.

    Parameters:
    ------------
    struct_img: np.ndarray
        a 3d image containing the LYSOSOME signal

    CY_object: np.ndarray boolean
        a 3d image containing the NU labels

    in_params: dict
        holds the needed parameters

    Returns:
    -------------
    tuple of:
        object
            mask defined boundaries of Lysosomes
        parameters: dict
            updated parameters in case any needed were missing
    """
    out_p= in_params.copy()

    ###################
    # PRE_PROCESSING
    ###################                         
    scaling_param =  [0]   
    struct_img = intensity_normalization(struct_img, scaling_param=scaling_param)
    out_p["intensity_norm_param"] = scaling_param

   # make a copy for post-post processing
    scaled_signal = struct_img.copy()

    med_filter_size = 3   
    # structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
    struct_img = median_filter_slice_by_slice( 
                                                                    struct_img,
                                                                    size=med_filter_size  )
    out_p["median_filter_size"] = med_filter_size 

    gaussian_smoothing_sigma = 1.
    gaussian_smoothing_truncate_range = 3.0
    struct_img = image_smoothing_gaussian_slice_by_slice(   struct_img,
                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                        truncate_range = gaussian_smoothing_truncate_range
                                                                                                    )
    out_p["gaussian_smoothing_sigma"] = gaussian_smoothing_sigma 
    out_p["gaussian_smoothing_truncate_range"] = gaussian_smoothing_truncate_range

    # log_img, d = log_transform( struct_img ) 
    # struct_img = intensity_normalization( log_img ,  scaling_param=[0] )  
    struct_img = intensity_normalization( struct_img ,  scaling_param=[0] )  


   ###################
    # CORE_PROCESSING
    ###################
    # dot and filiment enhancement - 2D

    ################################
    ## PARAMETERS for this step ##
    s2_param = [[5,0.09], [2.5,0.07], [1,0.01]]
    ################################
    bw_spot = dot_2d_slice_by_slice_wrapper(struct_img, s2_param)

    ################################
    ## PARAMETERS for this step ##
    f2_param = [[1, 0.15]]
    ################################
    bw_filament = filament_2d_wrapper(struct_img, f2_param)

    bw = np.logical_or(bw_spot, bw_filament)

    out_p["s2_param"] = s2_param 
    out_p["f2_param"] = f2_param 
    ###################
    # POST_PROCESSING
    ###################

    # 2D cleaning
    hole_max = 1600  
    # discount z direction
    struct_obj = aicssegmentation.core.utils.hole_filling(struct_obj, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 
    out_p['hole_max'] = hole_max


    # # 3D
    # cleaned_img = remove_small_objects(removed_holes>0, 
    #                                                             min_size=width, 
    #                                                             connectivity=1, 
    #                                                             in_place=False)

    small_object_max = 15
    struct_obj = aicssegmentation.core.utils.size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= small_object_max**3, 
                                                            method = "3D", #"slice_by_slice" ,
                                                            connectivity=1)
    out_p['small_object_max'] = small_object_max

                
    retval = (struct_obj,  out_p)
    return retval


# WORKFLOW #2 - WIP
as per 6/22 CellProfiler pipeline from MCZ
 > Speckles: A speckle is an area of enhanced intensity relative to its immediate neighborhood. The module enhances speckles using a white tophat filter, which is the image minus the morphological grayscale opening of the image. The opening operation first suppresses the speckles by applying a grayscale erosion to reduce everything within a given radius to the lowest value within that radius, then uses a grayscale dilation to restore objects larger than the radius to an approximation of their former shape. The white tophat filter enhances speckles by subtracting the effects of opening from the original image. 


## summary of steps

INPUT
- channel 1
- CYTO mask
PRE-PROCESSING
-   median filter, 2 pix window
- gaussian  Filter window 10

CORE-PROCESSING
- "BIG"
  - enhance speckles 
    - feature size: 40 pix
    - speed/accurace: slow
  - identify primary objects "BIG"
    - adaptive Otsu
      - diameter: (10,100)
      - three classes
        - middle intensity is foreground
        - threshold smoothing scale: 1.34
        - threshold correction factor: .7
        - threshold bounds: (0.08197, 1)
        - adaptive window: 20 pixels
- "SMALL"
  - enhance speckles 
    - feature size: 10 pix
    - speed/accurace: slow
- identify primary objects "SMALL"
  - adaptive Sauvola
    - diameter: (2,10)
    - threshold smoothing scale: 1.34
    - threshold correction factor: .75
    - threshold bounds: (0.06, 1)
    - adaptive window: 20 pixels

- POST-PROCESSING
  - embedded in CORE

OUTPUT
- mask of LYSOSOME




### INPUT

In [11]:

###################
# INPUT
###################
struct_img_raw = img_data[1,:,:,:].copy()

# DEFAULT PARAMETERS:
intensity_norm_param = [3.5, 15] # from Allen Cell Segmenter LAMP1  workflow
scaling_param = [0]
gaussian_smoothing_sigma = 1.
gaussian_smoothing_truncate_range = 3.0
dot_2d_sigma = 2
dot_2d_sigma_extra = 1
dot_2d_cutoff = 0.025
min_area = 10
low_level_min_size =  100

med_filter_size =2  

gaussian_smoothing_sigma = 10
gaussian_smoothing_truncate_range = 3.0

aicssegmentation.core.pre_processing_utils.suggest_normalization_param(struct_img_raw) #  [0., 23]


mean intensity of the stack: 641.5999045901829
the standard deviation of intensity of the stack: 2144.942904845627
0.9999 percentile of the stack intensity is: 49648.38039997965
minimum intensity of the stack: 0
maximum intensity of the stack: 65535
suggested upper range is 23.0, which is 49975.2867160396
suggested lower range is 0.0, which is 641.5999045901829
So, suggested parameter for normalization is [0.0, 23.0]
To further enhance the contrast: You may increase the first value (may loss some dim parts), or decrease the second value(may loss some texture in super bright regions)
To slightly reduce the contrast: You may decrease the first value, or increase the second value


### PRE-PROCESSING

In [22]:
###################
# PRE_PROCESSING
###################
# Linear-ish smoothing
raw_lysosomes = intensity_normalization( struct_img_raw ,  scaling_param=scaling_param)

# structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
struct_img = median_filter_slice_by_slice( 
                                                                raw_lysosomes,
                                                                size=med_filter_size  )


structure_img_smooth = image_smoothing_gaussian_slice_by_slice(   struct_img,
                                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                                        truncate_range=gaussian_smoothing_truncate_range,
                                                                                                                    )

# log_img, d = log_transform( structure_img_smooth ) 
# struct_img = intensity_normalization(  log_img  ,  scaling_param=[0] )  

struct_img = structure_img_smooth

intensity normalization: min-max normalization with NO absoluteintensity upper bound


### CORE PROCESSING

In [23]:
###################
# CORE_PROCESSING
###################
def _enhance_speckles(image, radius, volumetric=True):
    radius = radius / 2
    if volumetric:
        selem = ball(radius)
    else:
        selem = disk(radius)     

    # if radius >10:
    #         minimum = scipy.ndimage.minimum_filter(image, footprint=selem)
    #         maximum = scipy.ndimage.maximum_filter(minimum, footprint=selem)
    #         result = data - maximum
    # else:
    result =  white_tophat(image)

    return result

def enhance_neurites(image, radius, volumetric = True):
    if volumetric:
        selem = ball(radius)
    else:
        selem = disk(radius)     
    white = white_tophat(image,selem)
    black = black_tophat(image, selem)
    result = image + white - black
    result[result > 1] = 1
    result[result < 0] = 0

    return result

    



In [26]:
# enhance spreckles - 40
big_struct_rad = 40
big_img = _enhance_speckles(struct_img.copy(),big_struct_rad, True)



In [27]:

# enhance spreckles - 10
sm_struct_rad = 20
sm_img = _enhance_speckles(struct_img.copy(),sm_struct_rad, True)


#adaptive_otsu(big_struct) # three class - middle foreground
# adaptive window- 20
#size: 10,100
# threshold smooth 1.34
# threshold correction 1
# threshold (0.1497,1)
#fill holes
# "adatpive" thresholds are NOT currently working... will use      
# "Masked Object Thresholding" - 3D
low_level_min_size = 10
bw_big, _bw_low_level = MO(big_img, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= 0.5, 
                                                return_object=True,
                                                dilate=True)
 # or this?                                               
#    struct_obj = struct_img > filters.threshold_otsu(struct_img)
#     threshold_value_log = threshold_otsu_log(struct_img)

#     threshold_factor = 0.9 #from cellProfiler
#     thresh_min = .1
#     thresh_max = 1.
#     threshold = min( max(threshold_value_log*threshold_factor, thresh_min), thresh_max)
#     struct_obj = struct_img > threshold


# enhance speckles
#   adaptive_sauvola(sm_struct) 
# adaptive window- 10
#size: 2,10
# threshold smooth 1.34
# threshold correction 1
# threshold (0.05,1)
#fill holes
# "adatpive" thresholds are NOT currently working... will use      
# "Masked Object Thresholding" - 3D
low_level_min_size = 2
bw_sm, _bw_low_level = MO(sm_img, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= 0.5, 
                                                return_object=True,
                                                dilate=True)
 # or this?                                               
#    struct_obj = struct_img > filters.threshold_otsu(struct_img)
#     threshold_value_log = threshold_otsu_log(struct_img)
#     threshold_factor = 0.9 #from cellProfiler
#     thresh_min = .1
#     thresh_max = 1.
#     threshold = min( max(threshold_value_log*threshold_factor, thresh_min), thresh_max)
#     struct_obj = struct_img > threshold



### POST-PROCESSING

In [28]:
###################
# POST_PROCESSING
###################


# 3D
# cleaned_img = remove_small_objects(removed_holes>0, 
#                                                             min_size=minArea, 
#                                                             connectivity=1, 
#                                                             in_place=False)
small_object_max = 10
cleaned_img_big = size_filter(bw_big, # wrapper to remove_small_objects which can do slice by slice
                                                         min_size= small_object_max**2,
                                                         method = "slice_by_slice",
                                                         connectivity=1)


#                                                             in_place=False)
small_object_max = 2
cleaned_img_sm = size_filter(bw_sm, # wrapper to remove_small_objects which can do slice by slice
                                                         min_size= small_object_max**2,
                                                         method = "slice_by_slice",
                                                         connectivity=1)



cleaned_img = np.logical_or(cleaned_img_big, cleaned_img_sm)



#### Visualize with `napari`
Visualize the first-pass segmentation and labeling with `napari`.

In [29]:

if viewer is None:
    viewer = napari.view_image(
        cleaned_img,
        scale=scale
    )
else: 
    viewer.add_image(
        cleaned_img,
        scale=scale
    )

viewer.scale_bar.visible = True
viewer.add_image(
    cleaned_img_big,
    scale=scale
)

viewer.add_image(
    big_img,
    scale=scale
)
viewer.add_image(
    cleaned_img_big,
    scale=scale
)
viewer.add_image(
    sm_img,
    scale=scale
)

<Image layer 'sm_img' at 0x16ef15fd0>

# DEFINE `infer_LYSOSOME_CP` function ( WIP)

In [ ]:
# copy this to base.py for easy import
# mangle so we can call from base.py
def infer_LYSOSOME_CP(struct_img: np.ndarray, CY_object: np.ndarray, in_params:dict) -> tuple:
    """
    Procedure to infer LYSOSOMES from linearly unmixed input as per CellProfiler procedure

    Parameters:
    ------------
    struct_img: np.ndarray
        a 3d image containing the SOMA signal

    CY_object: np.ndarray boolean
        a 3d image containing the NU labels
        TODO:  perform the mask, flag?

    in_params: dict
        holds the needed parameters

    Returns:
    -------------
    tuple of:
        object
            mask defined boundaries of SOMA
        label
            label (could be more than 1)
        parameters: dict
            updated parameters in case any needed were missing
    
    """
    out_p= in_params.copy()

    ###################
    # PRE_PROCESSING
    ###################                         
    #TODO: replace params below with the input params
    scaling_param =  [0]   
    struct_img = intensity_normalization(struct_img, scaling_param=scaling_param)
    out_p["intensity_norm_param"] = scaling_param


    # 2D smoothing
    # make a copy for post-post processing
    scaled_signal = struct_img.copy()

    med_filter_size = 9   
    # structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
    struct_img = median_filter_slice_by_slice( 
                                                                    struct_img,
                                                                    size=med_filter_size  )
    out_p["median_filter_size"] = med_filter_size 

    gaussian_smoothing_sigma = 3.
    gaussian_smoothing_truncate_range = 3.0
    struct_img = image_smoothing_gaussian_slice_by_slice(   struct_img,
                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                        truncate_range = gaussian_smoothing_truncate_range
                                                                                                    )
    out_p["gaussian_smoothing_sigma"] = gaussian_smoothing_sigma 
    out_p["gaussian_smoothing_truncate_range"] = gaussian_smoothing_truncate_range

    #    edges = filters.scharr(struct_img)
    # struct_img, d = log_transform( struct_img ) 
    # struct_img = intensity_normalization(  struct_img,  scaling_param=[0] )
    ###################
    # CORE_PROCESSING
    ###################
    # "Masked Object Thresholding" - 3D
    local_adjust = 0.25
    low_level_min_size = 100
    struct_obj, _bw_low_level = MO(struct_img, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= local_adjust, 
                                                return_object=True,
                                                dilate=True)
    out_p["local_adjust"] = local_adjust 
    out_p["low_level_min_size"] = low_level_min_size 

    ###################
    # POST_PROCESSING
    ###################
    # 3D cleaning

    hole_max = 80  
    # discount z direction
    struct_obj = aicssegmentation.core.utils.hole_filling(struct_obj, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 
    out_p['hole_max'] = hole_max

    small_object_max = 35
    struct_obj = aicssegmentation.core.utils.size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= small_object_max**3, 
                                                            method = "3D", #"slice_by_slice" 
                                                            connectivity=1)
    out_p['small_object_max'] = small_object_max

    # 3D cleaning
    struct_obj = aicssegmentation.core.utils.hole_filling(struct_obj, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 
    struct_obj = aicssegmentation.core.utils.size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= small_object_max**3, 
                                                            method = "3D", #"slice_by_slice" 
                                                            connectivity=1)
    masked_labels_out = watershed(
                connectivity=np.ones((3, 3,3), bool),
                image=np.abs(ndi.sobel(struct_img)),
                markers=NU_labels,
                mask= np.logical_or(struct_obj, NU_labels == keep_label),
                )
                

    retval = (struct_obj,  masked_labels_out, out_p)
    return retval

# TEST `infer_LYSOSOMES` function

In [30]:
chan_name = 'nuclei'
out_path = data_path / "inferred_objects" 
object_name = 'CY_object'
CY_object, meta_dict_t = read_input_image( out_path/ f"{object_name}.ome.tiff" )


###################
# INPUT
###################
struct_img_raw = img_data[1,:,:,:].copy()

# DEFAULT PARAMETERS:
intensity_norm_param = [3.5, 15] # from Allen Cell Segmenter LAMP1  workflow
scaling_param = [0]
gaussian_smoothing_sigma = 1.
gaussian_smoothing_truncate_range = 3.0
dot_2d_sigma = 2
dot_2d_sigma_extra = 1
dot_2d_cutoff = 0.025
min_area = 10
low_level_min_size =  100

med_filter_size =3  

gaussian_smoothing_sigma = 1.3
gaussian_smoothing_truncate_range = 3.0

aicssegmentation.core.pre_processing_utils.suggest_normalization_param(struct_img_raw) #  [0., 23]



LY_object, out_p =  infer_LYSOSOMES(raw_soma_linear.copy(), CY_object, default_params) 
# TODO:  make export ome_tiff export:   XX_object, XX_label, XX_signal
# need to bapck all the objects into a single TIFF


chan_name = 'nuclei'
out_path = data_path / "inferred_objects" 
object_name = 'LY_object'

LY_object_filen = export_ome_tiff(LY_object, meta_dict, object_name, str(out_path)+"/", curr_chan=0)


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound


UnboundLocalError: local variable 'struct_obj' referenced before assignment

In [ ]:

viewer2.add_image(
    raw_soma_linear,
    scale=scale 
)

viewer2.add_image(
    SO_object,
    scale=scale
)
viewer2.scale_bar.visible = True

viewer2.add_labels(
    SO_label,
    scale=scale 
)
